In [1]:
import numpy as np
from Bio import pairwise2, SeqIO
from enum import Enum

In [13]:
class op(Enum):
    GAP_A = 1
    GAP_B = 2
    MA_MM = 3
    
def nw(seqA_str, seqB_str, score_mtx, gap_score = 0):
    na = len(seqA_str)
    nb = len(seqB_str)

    score_mtx = np.zeros((na + 1, nb + 1))
    score_mtx[:,0] = np.linspace(0, -na, na + 1)
    score_mtx[0,:] = np.linspace(0, -nb, nb + 1)

    tabla_sol = np.repeat(op.GAP_A, (na+1) * (nb+1)).reshape(na+1, nb+1)
    tabla_sol[:,0] = op.GAP_B
    tabla_sol[0,:] = op.GAP_A
    
    t = np.zeros(3)
    for i in range(na):
        for j in range(nb): 
            if seqA_str[i] == seqB_str[j]:
                t[0] = score_mtx[i,j] + 1
            else:
                t[0] = score_mtx[i,j] - 0
            t[1] = score_mtx[i,j+1] - gap_score
            t[2] = score_mtx[i+1,j] - gap_score
            tmax = np.max(t)
            score_mtx[i+1,j+1] = tmax
            if t[0] == tmax:
                tabla_sol[i+1,j+1] = op.MA_MM
            elif t[1] == tmax:
                tabla_sol[i+1,j+1] = op.GAP_B
            elif t[2] == tmax:
                tabla_sol[i+1,j+1] = op.GAP_A

    aln_a,aln_b = traceback(tabla_sol,seqA_str,seqB_str)
    
    return aln_a,aln_b,score_mtx[na,nb]

def traceback(tabla_sol, seqA, seqB):
    i = len(seqA)
    j = len(seqB)
    aln_a = []
    aln_b = []
    while i > 0 or j > 0:
        if tabla_sol[i,j] == op.MA_MM:
            aln_a.append(seqA[i-1])
            aln_b.append(seqB[j-1])
            i -= 1
            j -= 1
        elif tabla_sol[i,j] == op.GAP_B:
            aln_a.append(seqA[i-1])
            aln_b.append('-')
            i -= 1
        elif tabla_sol[i,j] == op.GAP_A:
            aln_a.append('-')
            aln_b.append(seqB[j-1])
            j -= 1

    aln_a = ''.join(aln_a)[::-1]
    aln_b = ''.join(aln_b)[::-1]
    return aln_a, aln_b

In [14]:
# Toma de input
f_aln = SeqIO.parse("ab.fasta", 'fasta')
fas_A = next(f_aln)
fas_B = next(f_aln)
seq_A = str(fas_A.seq)
seq_B = str(fas_B.seq)
# Confirme el score
score_mtx_1 = []
aln_a, aln_b, score = nw(seq_A, seq_B, score_mtx_1)
score

157.0

In [15]:
# Visualice el alineamiento. Recuerde que no es necesario que sea idéntico al de Biopython
str_aln_a = str().join(aln_a)
str_aln_b = str().join(aln_b)

print(str_aln_a[300:])
print(str_aln_b[300:])

CAACCCGGGGACAGAGTACTGGCCGCGGATTACGACGGA-AACCCGGTTTATACCGACTTCATCATGTTCAA
---CCC---GAC--ATTA-T-----C--TTTA--A-GGATGA---GG-----A--GA----A-CA----C-G


In [16]:
# Toma de input
f_aln = SeqIO.parse("ae.fasta", 'fasta')
fas_A = next(f_aln)
fas_E = next(f_aln)

seq_A = str(fas_A.seq)
seq_E = str(fas_E.seq)

In [17]:
# Confirme el score
aln_a, aln_e, score = nw(seq_A, seq_E, score_mtx_1)
score

312.0

In [18]:
# Visualice el alineamiento. Recuerde que no es necesario que sea idéntico al de Biopython
str_aln_a = str().join(aln_a)
str_aln_e = str().join(aln_e)

print(str_aln_a[300:])
print(str_aln_e[300:])

GACAGAGTACTGGCCGCGGATTACG-ACGGAAACCCGGTTTATACCGACTTCATCATGTTCAA
GACAGGGTCCTGGCAGCAGATGA-GAACGGAAACCCGATTTACACCGAC-TC--C---TT---
